<a href="https://colab.research.google.com/github/i-ganza007/PacMan_Formative/blob/main/MLPPOLICY_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -q --force-reinstall "numpy==1.26.4" "scipy<1.13" "scikit-learn<1.6" "matplotlib<3.9"

!apt-get update -qq && apt-get install -y -qq swig cmake libopenmpi-dev zlib1g-dev

!pip install -q \
    "gymnasium[box2d,atari,accept-rom-license]" \
    "stable-baselines3[extra]>=2.0.0" \
    "ale-py" \
    "torch>=2.0" \
    "tensorflow<2.17" \
    "keras<3.0"

!pip cache purge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 84.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 47.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 103.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 112.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 85.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.1 MB/s eta 0:00:00
 

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')

import ale_py
import gymnasium as gym
gym.register_envs(ale_py)
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd
import numpy as np
import random
import torch
import gc

In [3]:
gc.collect()
torch.cuda.empty_cache()

In [5]:
CONFIG_ID = 2

configs = [
    {"lr": 0.00035, "gamma": 0.99,  "batch": 12, "eps_start": 1.0, "eps_end": 0.03, "eps_decay": 0.25},
    {"lr": 0.00075, "gamma": 0.97,  "batch": 20, "eps_start": 1.0, "eps_end": 0.07, "eps_decay": 0.15},
    {"lr": 0.00008, "gamma": 0.992, "batch": 10, "eps_start": 1.0, "eps_end": 0.02, "eps_decay": 0.3},
    {"lr": 0.00045, "gamma": 0.985, "batch": 18, "eps_start": 1.0, "eps_end": 0.04, "eps_decay": 0.2},
    {"lr": 0.00028, "gamma": 0.98,  "batch": 14, "eps_start": 1.0, "eps_end": 0.015, "eps_decay": 0.18},
    {"lr": 0.00055, "gamma": 0.99,  "batch": 10, "eps_start": 1.0, "eps_end": 0.05, "eps_decay": 0.22},
    {"lr": 0.0009,  "gamma": 0.975, "batch": 12, "eps_start": 1.0, "eps_end": 0.08, "eps_decay": 0.12},
    {"lr": 0.00012, "gamma": 0.996, "batch": 16, "eps_start": 1.0, "eps_end": 0.01, "eps_decay": 0.28},
    {"lr": 0.0004,  "gamma": 0.988, "batch": 15, "eps_start": 1.0, "eps_end": 0.035, "eps_decay": 0.19},
    {"lr": 0.00018, "gamma": 0.993, "batch": 13, "eps_start": 1.0, "eps_end": 0.02, "eps_decay": 0.16},
]


# Experiment settings
CONFIG_START = 5
CONFIG_END = 10
TOTAL_TIMESTEPS = 100_000
N_EVAL_EPISODES = 10

# Results storage
results = []

print(f"Running Configurations {CONFIG_START} to {CONFIG_END}")
print(f"Total timesteps per config: {TOTAL_TIMESTEPS:,}")




Running Configurations 5 to 10
Total timesteps per config: 100,000


In [12]:
from datetime import datetime
import shutil
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback


In [13]:
for config_id in range(CONFIG_START, CONFIG_END + 1):
    
    print(f"CONFIG {config_id} - Starting Training")
    
    
    cfg = configs[config_id - 1]
    
    print(f"Hyperparameters:")
    print(f"  • Learning Rate: {cfg['lr']}")
    print(f"  • Gamma: {cfg['gamma']}")
    print(f"  • Batch Size: {cfg['batch']}")
    print(f"  • Epsilon: {cfg['eps_start']} → {cfg['eps_end']} (decay: {cfg['eps_decay']})")
    
    # Clear GPU memory
    gc.collect()
    torch.cuda.empty_cache()
    
    # Create environment
    env = gym.make("ALE/SpaceInvaders-v5", frameskip=4)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, n_stack=4)
    
    # Create model
    log_dir = f"/kaggle/working/logs/config_{config_id}/"
    os.makedirs(log_dir, exist_ok=True)
    
    model = DQN(
        "MlpPolicy",
        env,
        learning_rate=cfg["lr"],
        gamma=cfg["gamma"],
        batch_size=cfg["batch"],
        buffer_size=5000,
        learning_starts=2000,
        exploration_initial_eps=cfg["eps_start"],
        exploration_final_eps=cfg["eps_end"],
        exploration_fraction=cfg["eps_decay"],
        target_update_interval=1000,
        train_freq=4,
        gradient_steps=1,
        tensorboard_log=log_dir,
        verbose=0,
        device="cuda"
    )
    
   
    
    # Callbacks
    checkpoint_callback = CheckpointCallback(
        save_freq=100_000,
        save_path=f"/kaggle/working/checkpoints/config_{config_id}/",
        name_prefix=f"dqn_config{config_id}"
    )
    
    eval_callback = EvalCallback(
        env,
        best_model_save_path=f"/kaggle/working/best_models/config_{config_id}/",
        log_path=f"/kaggle/working/eval_logs/config_{config_id}/",
        eval_freq=20_000,
        deterministic=True,
        render=False,
        verbose=0
    )
    
    # Train
    start_time = datetime.now()
    try:
        model.learn(
            total_timesteps=TOTAL_TIMESTEPS,
            tb_log_name=f"Config{config_id}_MlpPolicy",
            log_interval=100,
            progress_bar=True,
            callback=[checkpoint_callback, eval_callback]
        )
        training_time = (datetime.now() - start_time).total_seconds()
        
        print(f"\n Training completed in {training_time/60:.2f} minutes")
        
        # Evaluate
        print(f" Evaluating model...")
        mean_reward, std_reward = evaluate_policy(
            model,
            env,
            n_eval_episodes=N_EVAL_EPISODES,
            deterministic=True
        )
        
        print(f"   Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
        
        # Save model
        model_path = f"/kaggle/working/trained_models/dqn_mlp_config{config_id}"
        os.makedirs("/kaggle/working/trained_models/", exist_ok=True)
        model.save(model_path)
        
        # Store results
        results.append({
            "config_id": config_id,
            "learning_rate": cfg["lr"],
            "gamma": cfg["gamma"],
            "batch_size": cfg["batch"],
            "eps_start": cfg["eps_start"],
            "eps_end": cfg["eps_end"],
            "eps_decay": cfg["eps_decay"],
            "mean_reward": mean_reward,
            "std_reward": std_reward,
            "training_time_minutes": training_time/60,
            "max_gpu_memory_gb": torch.cuda.max_memory_allocated()/1024**3,
            "status": "success"
        })
        
    except Exception as e:
        print(f"\n Training failed: {str(e)}")
        results.append({
            "config_id": config_id,
            "learning_rate": cfg["lr"],
            "gamma": cfg["gamma"],
            "batch_size": cfg["batch"],
            "eps_start": cfg["eps_start"],
            "eps_end": cfg["eps_end"],
            "eps_decay": cfg["eps_decay"],
            "mean_reward": None,
            "std_reward": None,
            "training_time_minutes": None,
            "max_gpu_memory_gb": None,
            "status": f"failed: {str(e)}"
        })
    
    finally:
        # Clean up
        env.close()
        del model
        gc.collect()
        torch.cuda.empty_cache()

CONFIG 5 - Starting Training
Hyperparameters:
  • Learning Rate: 0.00028
  • Gamma: 0.98
  • Batch Size: 14
  • Epsilon: 1.0 → 0.015 (decay: 0.18)


Output()


 Training completed in 16.45 minutes
 Evaluating model...
   Mean Reward: 180.00 ± 0.00
CONFIG 6 - Starting Training
Hyperparameters:
  • Learning Rate: 0.00055
  • Gamma: 0.99
  • Batch Size: 10
  • Epsilon: 1.0 → 0.05 (decay: 0.22)


Output()


 Training completed in 13.81 minutes
 Evaluating model...
   Mean Reward: 0.00 ± 0.00
CONFIG 7 - Starting Training
Hyperparameters:
  • Learning Rate: 0.0009
  • Gamma: 0.975
  • Batch Size: 12
  • Epsilon: 1.0 → 0.08 (decay: 0.12)


...


 Training completed in 16.61 minutes
 Evaluating model...
   Mean Reward: 281.50 ± 76.52
CONFIG 9 - Starting Training
Hyperparameters:
  • Learning Rate: 0.0004
  • Gamma: 0.988
  • Batch Size: 15
  • Epsilon: 1.0 → 0.035 (decay: 0.19)


Output()


 Training completed in 14.57 minutes
 Evaluating model...
   Mean Reward: 0.00 ± 0.00
CONFIG 10 - Starting Training
Hyperparameters:
  • Learning Rate: 0.00018
  • Gamma: 0.993
  • Batch Size: 13
  • Epsilon: 1.0 → 0.02 (decay: 0.16)


Output()


 Training completed in 15.01 minutes
 Evaluating model...
   Mean Reward: 135.00 ± 0.00


In [14]:
df_results = pd.DataFrame(results)
csv_filename = "/kaggle/working/experiment_results_configs_5_to_10.csv"
df_results.to_csv(csv_filename, index=False)

print(f" Results saved to: {csv_filename}\n")
print(df_results.to_string())

# Display results as a nice table
print("RESULTS SUMMARY TABLE")
display(df_results)


 Results saved to: /kaggle/working/experiment_results_configs_5_to_10.csv

   config_id  learning_rate  gamma  batch_size  eps_start  eps_end  eps_decay  mean_reward  std_reward  training_time_minutes  max_gpu_memory_gb   status
0          5        0.00028  0.980          14        1.0    0.015       0.18        180.0    0.000000              16.452810           0.603278  success
1          6        0.00055  0.990          10        1.0    0.050       0.22          0.0    0.000000              13.805970           1.081017  success
2          7        0.00090  0.975          12        1.0    0.080       0.12          0.0    0.000000              13.472208           1.082519  success
3          8        0.00012  0.996          16        1.0    0.010       0.28        281.5   76.519605              16.608277           1.085523  success
4          9        0.00040  0.988          15        1.0    0.035       0.19          0.0    0.000000              14.566556           1.085523  success
5

,config_id,learning_rate,gamma,batch_size,eps_start,eps_end,eps_decay,mean_reward,std_reward,training_time_minutes,max_gpu_memory_gb,status
0,5,0.00028,0.980,14,1.0,0.015,0.18,180.0,0.000000,16.452810,0.603278,success
1,6,0.00055,0.990,10,1.0,0.050,0.22,0.0,0.000000,13.805970,1.081017,success
2,7,0.00090,0.975,12,1.0,0.080,0.12,0.0,0.000000,13.472208,1.082519,success
3,8,0.00012,0.996,16,1.0,0.010,0.28,281.5,76.519605,16.608277,1.085523,success
4,9,0.00040,0.988,15,1.0,0.035,0.19,0.0,0.000000,14.566556,1.085523,success
5,10,0.00018,0.993,13,1.0,0.020,0.16,135.0,0.000000,15.011211,1.085523,success


In [15]:
# CELL 5: Find and Save Best Model
successful_results = df_results[df_results['status'] == 'success']

if not successful_results.empty:
    best_idx = successful_results['mean_reward'].idxmax()
    best_config = successful_results.loc[best_idx]
    
    print(" BEST MODEL")
    print(f"Config ID: {best_config['config_id']}")
    print(f"Mean Reward: {best_config['mean_reward']:.2f} ± {best_config['std_reward']:.2f}")
    print(f"Learning Rate: {best_config['learning_rate']}")
    print(f"Gamma: {best_config['gamma']}")
    print(f"Batch Size: {best_config['batch_size']}")
    print(f"Training Time: {best_config['training_time_minutes']:.2f} minutes")
    
    # Copy best model to submission location
    best_config_id = int(best_config['config_id'])
    source_path = f"./trained_models/dqn_mlp_config{best_config_id}.zip"
    best_model_path = "dqn_model_MLP_BEST.zip"
    
    if os.path.exists(source_path):
        shutil.copy(source_path, best_model_path)
        print(f"\n Best model copied to: {best_model_path}")
    
    # Also create a submission-ready version
    submission_path = "dqn_model.zip"
    shutil.copy(source_path, submission_path)
    print(f"Submission-ready model: {submission_path}")
else:
    print("\n No successful training runs!")

print("EXPERIMENT COMPLETED!")


 BEST MODEL
Config ID: 8
Mean Reward: 281.50 ± 76.52
Learning Rate: 0.00012
Gamma: 0.996
Batch Size: 16
Training Time: 16.61 minutes

 Best model copied to: dqn_model_MLP_BEST.zip
Submission-ready model: dqn_model.zip
EXPERIMENT COMPLETED!
